## Simple Binary Classification with defaults

In this notebook we will train a Wide and Deep model and simply a "Deep" model using the well known adult dataset

In [1]:
import numpy as np
import pandas as pd
import torch

from pytorch_widedeep.preprocessing import WidePreprocessor, TabPreprocessor
from pytorch_widedeep.training import Trainer
from pytorch_widedeep.models import Wide, TabMlp, WideDeep
from pytorch_widedeep.metrics import Accuracy, Precision
from pytorch_widedeep.datasets import load_adult

/Users/javierrodriguezzaurin/.pyenv/versions/3.8.12/envs/wd38/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df = load_adult(as_frame=True)
df.head()

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K


In [3]:
# For convenience, we'll replace '-' with '_'
df.columns = [c.replace("-", "_") for c in df.columns]
# binary target
df["income_label"] = (df["income"].apply(lambda x: ">50K" in x)).astype(int)
df.drop("income", axis=1, inplace=True)
df.head()

,age,workclass,fnlwgt,education,educational_num,marital_status,occupation,relationship,race,gender,capital_gain,capital_loss,hours_per_week,native_country,income_label
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,0
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,0
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,1
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,1
4,18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,0


In [4]:
df.drop(["fnlwgt", "educational_num"], axis=1, inplace=True)

### Preparing the data

In [5]:
# Define wide, crossed and deep tabular columns
wide_cols = [
    "workclass",
    "education",
    "marital_status",
    "occupation",
    "relationship",
    "race",
    "gender",
    "native_country",
]
crossed_cols = [("education", "occupation"), ("native_country", "occupation")]

In [6]:
cat_embed_cols = [
    "workclass",
    "education",
    "marital_status",
    "occupation",
    "relationship",
    "race",
    "gender",
    "capital_gain",
    "capital_loss",
    "native_country",
]
continuous_cols = ["age", "hours_per_week"]

In [7]:
cat_embed_cols

['workclass',
 'education',
 'marital_status',
 'occupation',
 'relationship',
 'race',
 'gender',
 'capital_gain',
 'capital_loss',
 'native_country']

In [8]:
# TARGET
target_col = "income_label"
target = df[target_col].values

let's see what the preprocessors do

In [9]:
# wide
wide_preprocessor = WidePreprocessor(wide_cols=wide_cols, crossed_cols=crossed_cols)
X_wide = wide_preprocessor.fit_transform(df)

In [10]:
# # wide_preprocessor has an attribute called encoding_dict with the encoding dictionary
# wide_preprocessor.encoding_dict

In [11]:
# deeptabular
tab_preprocessor = TabPreprocessor(
    embed_cols=cat_embed_cols, continuous_cols=continuous_cols
)
X_tab = tab_preprocessor.fit_transform(df)

In [12]:
# check the docs to understand the useful attributes that the tab_preprocessor has. For example,
# as well as an encoding dictionary, tab_preprocessor has an attribute called cat_embed_input
# that specifies the categortical columns that will be represented as embeddings, the number
# of different categories per feature, and the dimension of the embeddings as defined by some
# of the internal rules of thumb that the preprocessor has (have a look to the docs)
tab_preprocessor.cat_embed_input

[('workclass', 9, 5),
 ('education', 16, 8),
 ('marital_status', 7, 5),
 ('occupation', 15, 7),
 ('relationship', 6, 4),
 ('race', 5, 4),
 ('gender', 2, 2),
 ('capital_gain', 123, 24),
 ('capital_loss', 99, 21),
 ('native_country', 42, 13)]

In [13]:
print(X_wide)
print(X_wide.shape)

[[  1  10  26 ...  61 103 328]
 [  1  11  27 ...  61 104 329]
 [  2  12  27 ...  61 105 330]
 ...
 [  1  11  28 ...  61 115 335]
 [  1  11  26 ...  61 115 335]
 [  7  11  27 ...  61 127 336]]
(48842, 10)


In [14]:
print(X_tab)
print(X_tab.shape)

[[ 1.          1.          1.         ...  1.         -0.99512893
  -0.03408696]
 [ 1.          2.          2.         ...  1.         -0.04694151
   0.77292975]
 [ 2.          3.          2.         ...  1.         -0.77631645
  -0.03408696]
 ...
 [ 1.          2.          3.         ...  1.          1.41180837
  -0.03408696]
 [ 1.          2.          1.         ...  1.         -1.21394141
  -1.64812038]
 [ 7.          2.          2.         ...  1.          0.97418341
  -0.03408696]]
(48842, 12)


### Defining the model

In [15]:
wide = Wide(input_dim=np.unique(X_wide).shape[0], pred_dim=1)
tab_mlp = TabMlp(
    column_idx=tab_preprocessor.column_idx,
    cat_embed_input=tab_preprocessor.cat_embed_input,
    cat_embed_dropout=0.1,
    continuous_cols=continuous_cols,
    mlp_hidden_dims=[400, 200],
    mlp_dropout=0.5,
    mlp_activation="leaky_relu",
)

Let's first find out how a linear model performs 

In [16]:
wide

Wide(
  (wide_linear): Embedding(809, 1, padding_idx=0)
)

Before being passed to the Trainer, the models need to be "constructed" with the ``WideDeep`` constructor class. For the particular case of the wide/linear model, not much really happens

In [17]:
lin_model = WideDeep(wide=wide)

In [18]:
lin_model

WideDeep(
  (wide): Wide(
    (wide_linear): Embedding(809, 1, padding_idx=0)
  )
)

In [19]:
lin_trainer = Trainer(
    model=lin_model,
    objective="binary",
    optimizers=torch.optim.AdamW(lin_model.parameters(), lr=0.01),
    metrics=[Accuracy, Precision],
)

In [20]:
lin_trainer.fit(X_wide=X_wide, target=target, n_epochs=4, batch_size=128, val_split=0.2)

valid: 100%|███████████████████████████████████████| 77/77 [00:00<00:00, 235.98it/s, loss=0.36, metrics={'acc': 0.8353, 'prec': 0.6976}]


Bear in mind that `wide` is a linear model where the non-linearities are captured via the crossed columns. For the crossed-columns to be effective one needs proper business knowledge. There is no magic formula to produce them

Let's have a look to the tabular model by itself

In [21]:
tab_model = WideDeep(deeptabular=tab_mlp)

In [22]:
tab_model

WideDeep(
  (deeptabular): Sequential(
    (0): TabMlp(
      (cat_and_cont_embed): DiffSizeCatAndContEmbeddings(
        (cat_embed): DiffSizeCatEmbeddings(
          (embed_layers): ModuleDict(
            (emb_layer_workclass): Embedding(10, 5, padding_idx=0)
            (emb_layer_education): Embedding(17, 8, padding_idx=0)
            (emb_layer_marital_status): Embedding(8, 5, padding_idx=0)
            (emb_layer_occupation): Embedding(16, 7, padding_idx=0)
            (emb_layer_relationship): Embedding(7, 4, padding_idx=0)
            (emb_layer_race): Embedding(6, 4, padding_idx=0)
            (emb_layer_gender): Embedding(3, 2, padding_idx=0)
            (emb_layer_capital_gain): Embedding(124, 24, padding_idx=0)
            (emb_layer_capital_loss): Embedding(100, 21, padding_idx=0)
            (emb_layer_native_country): Embedding(43, 13, padding_idx=0)
          )
          (embedding_dropout): Dropout(p=0.1, inplace=False)
        )
        (cont_norm): BatchNorm1d(2, ep

You can see how the `WideDeep` class has added a final prediction layer that collects the activations from the last layer of the model and plugs them into the output neuron. If this was a multiclass classification problem, the prediction dimension (i.e. the size of that final layer) needs to be specified via the `pred_dim` when instantiating the `WideDeep` class, as we will see later

In [23]:
tab_trainer = Trainer(
    model=tab_model,
    objective="binary",
    optimizers=torch.optim.AdamW(tab_model.parameters(), lr=0.001),
    metrics=[Accuracy, Precision],
)

In [24]:
tab_trainer.fit(X_tab=X_tab, target=target, n_epochs=4, batch_size=128, val_split=0.2)

valid: 100%|██████████████████████████████████████| 77/77 [00:00<00:00, 169.21it/s, loss=0.323, metrics={'acc': 0.8651, 'prec': 0.7673}]


The best result I ever obtained with `LightGBM` on this dataset is 0.8782...so we are pretty close.

Let's combine the `wide` and `tab_mlp` components see if it helps

In [25]:
wide = Wide(input_dim=np.unique(X_wide).shape[0], pred_dim=1)
tab_mlp = TabMlp(
    column_idx=tab_preprocessor.column_idx,
    cat_embed_input=tab_preprocessor.cat_embed_input,
    cat_embed_dropout=0.1,
    continuous_cols=continuous_cols,
    mlp_hidden_dims=[400, 200],
    mlp_dropout=0.5,
    mlp_activation="leaky_relu",
)
wd_model = WideDeep(wide=wide, deeptabular=tab_mlp)

In [26]:
wd_trainer = Trainer(
    model=wd_model,
    objective="binary",
    optimizers=torch.optim.AdamW(wd_model.parameters(), lr=0.001),
    metrics=[Accuracy, Precision],
)

In [27]:
wd_trainer.fit(
    X_wide=X_wide, X_tab=X_tab, target=target, n_epochs=4, batch_size=128, val_split=0.2
)

valid: 100%|███████████████████████████████████████| 77/77 [00:00<00:00, 150.39it/s, loss=0.339, metrics={'acc': 0.8434, 'prec': 0.663}]


For this particular case, the combination of both did not lead to better results that using just the tab_mlp model. 

Note that we have use a `TabMlp` model, but we could use any other model in the library using the same syntax

In [28]:
from pytorch_widedeep.models import TabTransformer

The parameters for the `TabTransformer` are this

```
column_idx: Dict[str, int],
cat_embed_input: Union[List[Tuple[str, int]], NoneType] = None,
cat_embed_dropout: float = 0.1,
use_cat_bias: bool = False,
cat_embed_activation: Union[str, NoneType] = None,
full_embed_dropout: bool = False,
shared_embed: bool = False,
add_shared_embed: bool = False,
frac_shared_embed: float = 0.25,
continuous_cols: Union[List[str], NoneType] = None,
cont_norm_layer: str = None,
embed_continuous: bool = False,
cont_embed_dropout: float = 0.1,
use_cont_bias: bool = True,
cont_embed_activation: Union[str, NoneType] = None,
input_dim: int = 32,
n_heads: int = 8,
use_qkv_bias: bool = False,
n_blocks: int = 4,
attn_dropout: float = 0.2,
ff_dropout: float = 0.1,
transformer_activation: str = 'gelu',
mlp_hidden_dims: Union[List[int], NoneType] = None,
mlp_activation: str = 'relu',
mlp_dropout: float = 0.1,
mlp_batchnorm: bool = False,
mlp_batchnorm_last: bool = False,
mlp_linear_first: bool = True,
```

Please, see the documentation for details on each one of them, for now let's see how one could use a `TabTransformer` model in a few lines of code

In [29]:
tab_transformer = TabTransformer(
    column_idx=tab_preprocessor.column_idx,
    cat_embed_input=tab_preprocessor.cat_embed_input,
    cat_embed_dropout=0.1,
    continuous_cols=continuous_cols,
    embed_continuous=True,
    cont_embed_dropout=0.2,
    use_cont_bias=True,
    cont_embed_activation="leaky_relu",
    n_heads=4,
    ff_dropout=0.2,
    mlp_dropout=0.5,
    mlp_activation="leaky_relu",
)

In [30]:
tab_model = WideDeep(deeptabular=tab_transformer)

In [31]:
tab_model

WideDeep(
  (deeptabular): Sequential(
    (0): TabTransformer(
      (cat_and_cont_embed): SameSizeCatAndContEmbeddings(
        (cat_embed): SameSizeCatEmbeddings(
          (embed): Embedding(325, 32, padding_idx=0)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (cont_norm): Identity()
        (cont_embed): ContEmbeddings(2, 32, embed_dropout=0.2, use_bias=True)
      )
      (cont_embed_act_fn): LeakyReLU(negative_slope=0.01, inplace=True)
      (transformer_blks): Sequential(
        (transformer_block0): TransformerEncoder(
          (attn): MultiHeadedAttention(
            (dropout): Dropout(p=0.2, inplace=False)
            (q_proj): Linear(in_features=32, out_features=32, bias=False)
            (kv_proj): Linear(in_features=32, out_features=64, bias=False)
            (out_proj): Linear(in_features=32, out_features=32, bias=False)
          )
          (ff): FeedForward(
            (w_1): Linear(in_features=32, out_features=128, bias=True)
            

In [32]:
tab_trainer = Trainer(
    model=tab_model,
    objective="binary",
    optimizers=torch.optim.AdamW(tab_model.parameters(), lr=0.001),
    metrics=[Accuracy, Precision],
)

In [33]:
tab_trainer.fit(X_tab=X_tab, target=target, n_epochs=1, batch_size=128, val_split=0.2)

valid: 100%|█████████████████████████████████████████| 77/77 [00:01<00:00, 44.97it/s, loss=0.302, metrics={'acc': 0.8637, 'prec': 0.75}]
